In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Selecting features for model performance

Another, more pragmatic, approach to decide on keeping individual or pairwise properties of features in the dataset or not is to select features based on how they affect model performance.

In [57]:
df = pd.read_csv("PimaIndians.csv")
df.head()

,pregnant,glucose,diastolic,triceps,insulin,bmi,family,age,test
0,1,89,66,23,94,28.1,0.167,21,negative
1,0,137,40,35,168,43.1,2.288,33,positive
2,3,78,50,32,88,31.0,0.248,26,positive
3,2,197,70,45,543,30.5,0.158,53,positive
4,1,189,60,23,846,30.1,0.398,59,positive


In [58]:
X = df.drop("test", axis = 1)

In [59]:
y = df["test"]

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

#### Pre-processing the data

To train a model on this data we should first perform a train - test split, and in this case also standardize the training feature dataset X_train to have a mean of zero and a variance of one.

In [62]:
from sklearn.preprocessing import StandardScaler

In [63]:
scaler = StandardScaler()

In [64]:
# Fit the scaler on the training features and transform these in one go
X_train_std = scaler.fit_transform(X_train)

#### Creating a logistic regression model

We can then create and fit a logistic regression model on this standardized training data. 

In [65]:
from sklearn.linear_model import LogisticRegression

In [66]:
lr = LogisticRegression()

In [67]:
# Fit the logistic regression model on the scaled training data
lr.fit(X_train_std, y_train)

LogisticRegression()

To see how the model performs on the test set we first scale these features with the .transform() method of the scaler that we just fit on the training set and then make our prediction. 

In [68]:
# Scale the test features
X_test_std = scaler.transform(X_test)

In [69]:
# Predict diabetes presence on the scaled test set
y_pred = lr.predict(X_test_std)

In [70]:
from sklearn.metrics import accuracy_score

In [71]:
lr.coef_[0]

array([ 0.2400702 ,  1.28616614, -0.03981669,  0.17499413, -0.14430102,
        0.43329563,  0.31921669,  0.35260952])

#### Inspecting the feature coefficients

However, when we look at the feature coefficients that the logistic regression model uses in its decision function, we'll see that some values are pretty close to zero. 

Since these coefficients will be multiplied with the feature values when the model makes a prediction, features with coefficients close to zero will contribute little to the end result. 

We can use the zip function to transform the output into a dictionary that shows which feature has which coefficient. 

In [16]:
# Prints accuracy metrics and feature coefficients
print("{0:.1%} accuracy on test set.".format(accuracy_score(y_test, y_pred))) 
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

76.9% accuracy on test set.
{'pregnant': 0.24, 'glucose': 1.29, 'diastolic': 0.04, 'triceps': 0.17, 'insulin': 0.14, 'bmi': 0.43, 'family': 0.32, 'age': 0.35}


We get almost 80% accuracy on the test set. 

#### Manual Recursive Feature Elimination

If we want to remove a feature from the initial dataset with as little effect on the predictions as possible, we should pick the one with the lowest coefficient, 'diastolic' in this case. The fact that we standardized the data first makes sure that we can compare the coefficients to one another.


Now we'll remove the feature with the lowest model coefficient from X, in this case we'll remove feature 

In [17]:
# Remove the feature with the lowest model coefficient--->>> 'diastolic'
X = df[['pregnant', 'glucose','triceps', 'insulin', 'bmi', 'family', 'age']]

In [18]:
# Performs a 25-75% train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [19]:
# Scales features and fits the logistic regression model
lr.fit(scaler.fit_transform(X_train), y_train)

LogisticRegression()

In [20]:
# Calculates the accuracy on the test set and prints coefficients
acc = accuracy_score(y_test, lr.predict(scaler.transform(X_test)))
print("{0:.1%} accuracy on test set.".format(acc)) 
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

80.6% accuracy on test set.
{'pregnant': 0.05, 'glucose': 1.24, 'triceps': 0.24, 'insulin': 0.2, 'bmi': 0.39, 'family': 0.34, 'age': 0.35}


We get almost 81% accuracy on the test set.

Now we'll remove 2 more features with the lowest model coefficients, which are features - 'pregnant' and 'insulin'

In [21]:
# Remove the 2 features with the lowest model coefficients---->>> 'pregnant', 'insulin'
X = df[['glucose', 'triceps', 'bmi', 'family', 'age']]

In [22]:
# Performs a 25-75% train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [23]:
# Scales features and fits the logistic regression model
lr.fit(scaler.fit_transform(X_train), y_train)

LogisticRegression()

In [24]:
# Calculates the accuracy on the test set and prints coefficients
acc = accuracy_score(y_test, lr.predict(scaler.transform(X_test)))
print("{0:.1%} accuracy on test set.".format(acc)) 
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

79.6% accuracy on test set.
{'glucose': 1.13, 'triceps': 0.25, 'bmi': 0.34, 'family': 0.34, 'age': 0.37}


We get almost 80% accuracy on the test set.

Now we'll only keep the feature with the highest coefficient which is 'glucose'

In [25]:
# Only keep the feature with the highest coefficient
X = df[['glucose']]

In [26]:
# Performs a 25-75% train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [27]:
# Scales features and fits the logistic regression model to the data
lr.fit(scaler.fit_transform(X_train), y_train)

LogisticRegression()

In [28]:
# Calculates the accuracy on the test set and prints coefficients
acc = accuracy_score(y_test, lr.predict(scaler.transform(X_test)))
print("{0:.1%} accuracy on test set.".format(acc)) 
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

75.5% accuracy on test set.
{'glucose': 1.28}


We get almost 76% accuracy on the test set with only one feature, that's not bad

#### Recursive Feature Elimination

We could repeat this process until we have the desired number of features remaining, but it turns out, there's a Scikit-learn function that does just that.

RFE for "Recursive Feature Elimination" is a feature selection algorithm that can be wrapped around any model that produces feature coefficients or feature importance values. 

In [142]:
from sklearn.feature_selection import RFE

Here, We can pass it the model we want to use and the number of features we want to select. While fitting to our data it will repeat a process where it first fits the internal model and then drops the feature with the weakest coefficient. It will keep doing this until the desired number of features is reached. 

If we set RFE's verbose parameter higher than zero we'll be able to see that features are dropped one by one while fitting. 

We could also decide to just keep the 2 features with the highest coefficients after fitting the model once, but this recursive method is safer, since dropping one feature will cause the other coefficients to change.

In [143]:
# Create the RFE with a LogisticRegression estimator and 3 features to select
rfe = RFE(estimator=LogisticRegression(), n_features_to_select=3, verbose=1)

In [144]:
X = df.drop("test", axis = 1)
y = df["test"]

In [145]:
# Performs a 25-75% train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [146]:
# Fit the scaler on the training features and transform these in one go
X_train_std = scaler.fit_transform(X_train)
# Scale the test features
X_test_std = scaler.transform(X_test)

In [147]:
# Fits the eliminator to the data
rfe.fit(X_train_std, y_train)


Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.


RFE(estimator=LogisticRegression(), n_features_to_select=3, verbose=1)

#### Inspecting the RFE results

Once RFE is done, by Using the zip function once more, we can also check out rfe's ranking_ attribute to see in which iteration a feature was dropped. Values of 1 mean that the feature was kept in the dataset until the end while high values mean the feature was dropped early on. 

In [148]:
# Print the features and their ranking (high = dropped early on)
print(dict(zip(X.columns, rfe.ranking_)))

{'pregnant': 5, 'glucose': 1, 'diastolic': 6, 'triceps': 3, 'insulin': 4, 'bmi': 1, 'family': 2, 'age': 1}


we can check the support_ attribute that contains True/False values to see which features were kept in the dataset. 

In [149]:
# Print the features that are not eliminated
print(X.columns[rfe.support_])

Index(['glucose', 'bmi', 'age'], dtype='object')


Finally, we can test the accuracy of the model with just three remaining features, 'glucose', 'bmi' and 'age', turns out the accuracy is still untouched at 81%. This means the other 5 features had little to no impact on the model an its predictions.

In [150]:
# Calculates the test set accuracy
acc = accuracy_score(y_test, rfe.predict(X_test_std))
print("{0:.1%} accuracy on test set.".format(acc)) 

81.0% accuracy on test set.


#### Tree-based feature selection

Some models will perform feature selection by design to avoid overfitting. One of those, is the random forest classifier. It's an ensemble model that will pass different, random, subsets of features to a number of decision trees. To make a prediction it will aggregate over the predictions of the individual trees.

While simple in design, random forests often manage to be highly accurate and avoid overfitting even with the default Scikit-learn settings.


By averaging how often features are used to make decisions inside the different decision trees, and taking into account whether these are important decisions near the root of the tree or less important decisions in the smaller branches of the tree, the random forest algorithm manages to calculate feature importance values.

### Random Forest

In [259]:
X = df.drop("test", axis = 1)
y = df["test"]

In [260]:
# Perform a 75% training and 25% test data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [261]:
from sklearn.ensemble import RandomForestClassifier

In [262]:
# Fit the random forest model to the training data
rf = RandomForestClassifier(random_state=0)

In [263]:
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [264]:
# Calculate the accuracy
acc = accuracy_score(y_test, rf.predict(X_test))

In [265]:
# Print the importances per feature
print(dict(zip(X.columns, rf.feature_importances_.round(2))))

{'pregnant': 0.07, 'glucose': 0.25, 'diastolic': 0.09, 'triceps': 0.09, 'insulin': 0.14, 'bmi': 0.12, 'family': 0.12, 'age': 0.13}


In [266]:
# Print accuracy
print("{0:.1%} accuracy on test set.".format(acc))

79.6% accuracy on test set.


#### Feature importance values

Feature importance values can be extracted from a trained model using the feature_importances_ attribute. 

Just like the coefficients produced by the logistic regressor, these feature importance values can be used to perform feature selection, since for unimportant features they will be close to zero. 

An advantage of these feature importance values over coefficients is that they are comparable between features by default, since they always sum up to one. Which means we don't have to scale our input data first.


#### Feature importance as a feature selector

We can use the feature importance values to create a True/False mask for features that meet a certain importance threshold. Then, we can apply that mask to our feature dataframe to implement the actual feature selection.

In [267]:
# Create a mask for features importances above the threshold
mask = rf.feature_importances_ > 0.12
mask

array([False,  True, False, False,  True, False, False,  True])

In [268]:
# Apply the mask to the feature dataset X
reduced_X = X.loc[:,mask]

In [269]:
# prints out the selected column names
print(reduced_X.columns)

Index(['glucose', 'insulin', 'age'], dtype='object')


#### RFE with random forests

Remember dropping one weak feature can make other features relatively more or less important. If you want to play safe and minimize the risk of dropping the wrong features, you should not drop all least important features at once but rather one by one. To do so we can once again wrap a Recursive Feature Eliminator, or RFE(), around our model. 

Here, we've reduced the number of features to 3 with no reduction in test set accuracy. However, training the model once for each feature we want to drop can result in too much computational overhead.

To speed up the process we can pass the "step" parameter to RFE(). Here, we've set it to 2 so that on each iteration the 2 least important features are dropped. 

In [270]:
# Set the feature eliminator to remove 2 features on each step
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=3, step = 2, verbose=1)

In [271]:
# Fit the model to the training data
rfe.fit(X_train, y_train)

Fitting estimator with 8 features.
Fitting estimator with 6 features.
Fitting estimator with 4 features.


RFE(estimator=RandomForestClassifier(), n_features_to_select=3, step=2,
    verbose=1)

Once the final model is trained, we can use the feature eliminator's .support_ attribute as a mask to print the remaining column names.

In [272]:
# Create a mask
mask = rfe.support_

In [273]:
# Apply the mask to the feature dataset X and print the result
reduced_X = X.loc[:, mask]

In [274]:
print(reduced_X.columns)

Index(['glucose', 'insulin', 'age'], dtype='object')


In [350]:
X = df[['glucose', 'insulin', 'age']]

In [351]:
# Perform a 75% training and 25% test data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.21, random_state=0)

In [352]:
# Fit the random forest model to the training data
rf = RandomForestClassifier(random_state=0)

In [353]:
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [354]:
# Calculate the accuracy
acc = accuracy_score(y_test, rf.predict(X_test))

In [355]:
# Print accuracy
print("{0:.1%} accuracy on test set.".format(acc))

80.7% accuracy on test set.
